# Step 2: ANALYZE

In [1]:
import pandas as pd
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0
import re
import pandas as pd
import numpy as np
import platform
import matplotlib.pyplot as plt
import json


In [2]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS as SW_en
from spacy.lang.es.stop_words import STOP_WORDS as SW_es

In [3]:
from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

In [4]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [5]:
#import nltk 
#from nltk.corpus import stopwords 
#set(stopwords.words('english'))


In [6]:
#!python -m spacy download en_core_web_md
#!python -m spacy download en_core_web_sm
#!python -m spacy download es_core_news_md

In [7]:
##################################333 PARAMETERS #####################

key_words_DA = ['data analyst', 'analisis de datos',
                      'data analytics', 'analitica de datos',
                      'business analyst', 'analisis de negocios',
                      'sql',
                      'python',
                      'business intelligence', 'inteligencia de negocios', ' bi ',
                      'excel', 'microsoft office', 'ms office',
                      'dashboard', 'kpi',
                      'reporting', 'informes',
                      'tableau', 'powerbi', 'power bi', 'microsoft bi', 'qlik', 'spotfire',
                      ]

key_words_DS = ['data analyst', 'analisis de datos',
                      'data analytics', 'analitica de datos',
                      'business analyst', 'analisis de negocios',
                      'sql',
                      'python',
                      'business intelligence', 'inteligencia de negocios', ' bi ',
                      'excel', 'microsoft office', 'ms office',
                      'dashboard', 'kpi',
                      'reporting', 'informes',
                      'tableau', 'powerbi', 'power bi', 'microsoft bi', 'qlik', 'spotfire',
                      ]

key_words_DE = ['data engineer', 'ingeniero de datos',
                       'sql', 'nosql', 'no sql', 'bases de datos no relacionales',
                       'python', ' r ', 'scala', 'java ', 'spark', 'hadoop', 'haddop' 'hive', 'impala', 'kafka',
                       'data wrangling', 'limpieza de datos',
                       'etl', 'extract, transform and load', 'map reduce', 'mapreduce',
                       'data acquisition', 'adquisicion de datos',
                       'big data', 'bigdata', 'bigquery',
                       'data warehouse',
                       ' aws ', 'azure',
                       'software engineering'
                       ]

dic_key_words = \
    {'data analyst'    : key_words_DA,
     'data scientist'  : key_words_DS,
     'data engineer'   : key_words_DE,
     'analista datos'  : key_words_DA,
     'cientifico datos': key_words_DS,
     'ingeniero datos' : key_words_DE
     }

In [72]:
def job_key_generator(job):
    job_key_s = ""
    job_name_splitted = job.split(" ")
    for string in job_name_splitted:
        job_key_s += string[0].upper()
        #job_key_s = job_key_s
    
    return job_key_s

## Getting csv

#### > PROFILE:

In [8]:
platform.system()

'Linux'

In [9]:
if platform.system() == 'Linux':
    df_profile = pd.read_csv(f'../data/raw/df_profile/df_profile_cristopherrojaslepe_2020-04-23 17.29.csv',
                             sep = ';',
                             encoding= 'utf8',
                             index_col = 0
                              )
    
elif platform.system() == 'Windows':
    df_profile = pd.read_csv(f'..\\data\\raw\\df_profile.csv',
                             sep = ';',
                             encoding= 'utf8'
                            )
    
df_profile

,info
profile,cristopherrojaslepe
headline,data scientist & analyst | python | pandas | n...
open new jobs,True
about,data scientist & analyst and industrial engine...
experiences,"['collection coordination analyst', 'real esta..."
years of experiences,"['1 yr 2 mos', '1 yr', '2 yrs 1 mo', '8 mos', ..."
total years,7
degrees,"['master', 'engineering - be']"
fields,"['data analytics', 'industrial engineering']"
skills,"['analytical skills', 'bash', 'business analys..."


In [10]:
## ONLY FOR JUPYTER !!!!!!!!!!
# df_profile.rename(columns={'Unnamed: 0':'type'},inplace=True)
# df_profile.set_index('type', inplace=True)

#### > JOBS:

In [12]:
#f'../data/raw/df_jobs_Madrid_2020-04-18 23.50.csv',
#f'..\\data\\raw\\df_jobs_Madrid_2020-04-18 23.50.csv'
if platform.system() == 'Linux':
    df_jobs_raw = pd.read_csv(f'../data/raw/df_jobs/df_jobs_Madrid_en_2020-04-20 01.38_465.csv',
                              sep = ';',
                              encoding= 'utf8',
                  )
elif platform.system() == 'Windows':
    df_jobs_raw = pd.read_csv(f'..\\data\\raw\\df_jobs\\df_jobs_Madrid_2020-04-20 01.38.csv',
                              sep = ';',
                              encoding= 'utf8',
                  )

In [13]:
df_jobs = df_jobs_raw

In [14]:
df_jobs.shape

(465, 18)

In [15]:
df_jobs.head(3)

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,Easy apply,Skills match,Seniority Level,Industry,Employment Type,Job Functions,Job info,Job Description
0,data analyst,Madrid,2020-04-20 01:38,1821611516,https://www.linkedin.com/jobs/view/1821611516/,Data Analyst Scala H/M,Experis IT,"Madrid, Comunidad de Madrid, España",5 days,2020-04-15 01:38,True,"{'yes': ['Data Analysis', 'Business Intelligen...",Mid-Senior level,Information Services Telecommunications Banking,Full-time,Information Technology Consulting Project Mana...,Posted Date\nPosted 5 days ago\nNumber of view...,"Experis IT, somos una compañía líder en soluci..."
1,data analyst,Madrid,2020-04-20 01:38,1811532919,https://www.linkedin.com/jobs/view/1811532919/,Finance Systems Business Analyst - GL/FA/AP/AR...,SystemsAccountants,Greater Madrid Metropolitan Area,1 week,2020-04-13 01:38,True,"{'yes': ['Business', 'Spanish', 'Accounting', ...",Mid-Senior level,Consumer Goods Consumer Services Financial Ser...,Full-time,Information Technology Finance Project Management,Posted Date\nPosted 1 week ago\nNumber of view...,A large company based near Madrid is currentl...
2,data analyst,Madrid,2020-04-20 01:38,1800343808,https://www.linkedin.com/jobs/view/1800343808/,Senior Business Development Manager - Cloud Ec...,Amazon Web Services (AWS),"Madrid, ES",3 weeks,2020-03-30 01:38,False,"{'yes': ['Statistics', 'SQL', 'Microsoft Excel...",Mid-Senior level,Computer Software Information Technology & Ser...,Full-time,Business Development Information Technology,Posted Date\nPosted 3 weeks ago\nNumber of vie...,Description\n\nWould you like to be part of a ...


In [16]:
# ONLY IN CASE TO CORRECT WRONG DF
#df_jobs.drop(columns='Job htmlJob name', inplace=True)

#df_jobs = df_jobs[['JOB TITLE', 'LOCATION', 'SEARCH DATETIME', 'Current Job Id',
#       'Job html', 'Job name', 'Company name', 'Company location',
#       'Posted date', 'Estimated post date', 'Easy apply', 'Skills match',
#       'Seniority Level', 'Industry', 'Employment Type', 'Job Functions',
#       'Job info']]

In [17]:
df_jobs[df_jobs['JOB TITLE']=='data analyst'].shape

(98, 18)

In [18]:
df_jobs[df_jobs['JOB TITLE']=='data scientist'].shape

(255, 18)

In [19]:
df_jobs[df_jobs['JOB TITLE']=='data engineer'].shape

(112, 18)

## Analizyng info

In [20]:
location = df_jobs.loc[0,'LOCATION']
location

'Madrid'

### > Cleaning jobs:

In [21]:
df_jobs.loc[0,'JOB TITLE':'Current Job Id']

JOB TITLE              data analyst
LOCATION                     Madrid
SEARCH DATETIME    2020-04-20 01:38
Current Job Id           1821611516
Name: 0, dtype: object

##### * JOB TITLE_ID

In [22]:
df_jobs['JOB TITLE_ID'] = df_jobs.apply(lambda x: x['JOB TITLE'] +"_"+ str(x['Current Job Id']), axis = 1 )

In [23]:
df_jobs.shape[0] 

465

In [60]:
len(df_jobs['JOB TITLE_ID'].unique())

462

In [25]:
pd.pivot_table(df_jobs,
               values="LOCATION",
               index=['JOB TITLE_ID'],
               aggfunc=np.count_nonzero        
              ).sort_values(by=['LOCATION'], ascending=False)

,LOCATION
JOB TITLE_ID,
data scientist_1796064556,2
data engineer_1807219706,2
data analyst_1800343808,2
data analyst_1298845523,1
data scientist_1808206378,1
...,...
data engineer_1809991992,1
data engineer_1809448415,1
data engineer_1808744900,1


In [26]:
df_jobs[df_jobs['Current Job Id']==1774326710]

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,Easy apply,Skills match,Seniority Level,Industry,Employment Type,Job Functions,Job info,Job Description,JOB TITLE_ID


In [27]:
df_jobs.shape[0] - len(df_jobs['JOB TITLE_ID'].unique())

3

In [28]:
df_jobs.drop_duplicates(subset='JOB TITLE_ID', keep='first', inplace = True)

In [29]:
df_jobs.shape

(462, 19)

##### * Current Job Id

In [30]:
len(df_jobs['Current Job Id'].unique())

403

In [31]:
pd.pivot_table(df_jobs,
               values="LOCATION",
               index=['Current Job Id'],
               aggfunc=np.count_nonzero        
              ).sort_values(by=['LOCATION'], ascending=False)

,LOCATION
Current Job Id,
1806494044,2
1828476880,2
1821384825,2
1818317791,2
1825643625,2
...,...
1800343808,1
1799852661,1
1797911270,1


In [71]:
unique_jobs = list(df_jobs['JOB TITLE'].unique())
unique_jobs

['data analyst', 'data scientist', 'data engineer']

In [80]:
pivot_ID_TITLES = pd.pivot_table(df_jobs,
               values="LOCATION",
               index=['Current Job Id'],
               columns=['JOB TITLE'],
               aggfunc=np.count_nonzero        
              ).reset_index()

pivot_ID_TITLES['Multiple Job Post'] = ""

for j in unique_jobs:
    pivot_ID_TITLES[j] = \
    pivot_ID_TITLES[j].apply(lambda x: job_key_generator(j) if x==1.0 else '')
    pivot_ID_TITLES['Multiple Job Post'] = pivot_ID_TITLES['Multiple Job Post'] + pivot_ID_TITLES[j] + '-'

pivot_ID_TITLES

JOB TITLE,Current Job Id,data analyst,data engineer,data scientist,Multiple Job Post
0,1276595125,,,DS,-DS--
1,1298845523,DA,,,DA---
2,1298846568,DA,,,DA---
3,1298846570,,,DS,-DS--
4,1328392365,,,DS,-DS--
...,...,...,...,...,...
398,1830255201,DA,,DS,DA-DS--
399,1831080577,,DE,,--DE-
400,1831081455,,,DS,-DS--
401,1831082025,,,DS,-DS--


In [81]:
df_jobs = df_jobs.merge(pivot_ID_TITLES[['Current Job Id','Multiple Job Post']],
                       on='Current Job Id',
                       how='left')

In [82]:
df_jobs[df_jobs['Current Job Id']==1645918674]

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,...,Job Functions,Job info,Job Description,JOB TITLE_ID,Info Language,da coincidences,da coincidences_list,PROFILE coincidences,PROFILE coincidences_list,Multiple Job Post
119,data scientist,Madrid,2020-04-20 01:38,1645918674,https://www.linkedin.com/jobs/view/1645918674/,Senior Data Analyst & Data Scientist,Ekkiden,"Madrid, ES",4 months,2019-12-30 01:38,...,Analyst Engineering Information Technology,Posted Date\nPosted 4 months ago\nNumber of vi...,This is Ekkiden\n\nAmbition and innovation are...,data scientist_1645918674,en,2,"[sql, python]",10,"[analytical skills, data analytics, data scien...",-DS--


In [33]:
df_jobs.drop_duplicates(subset='Current Job Id', keep='first')

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,Easy apply,Skills match,Seniority Level,Industry,Employment Type,Job Functions,Job info,Job Description,JOB TITLE_ID
0,data analyst,Madrid,2020-04-20 01:38,1821611516,https://www.linkedin.com/jobs/view/1821611516/,Data Analyst Scala H/M,Experis IT,"Madrid, Comunidad de Madrid, España",5 days,2020-04-15 01:38,True,"{'yes': ['Data Analysis', 'Business Intelligen...",Mid-Senior level,Information Services Telecommunications Banking,Full-time,Information Technology Consulting Project Mana...,Posted Date\nPosted 5 days ago\nNumber of view...,"Experis IT, somos una compañía líder en soluci...",data analyst_1821611516
1,data analyst,Madrid,2020-04-20 01:38,1811532919,https://www.linkedin.com/jobs/view/1811532919/,Finance Systems Business Analyst - GL/FA/AP/AR...,SystemsAccountants,Greater Madrid Metropolitan Area,1 week,2020-04-13 01:38,True,"{'yes': ['Business', 'Spanish', 'Accounting', ...",Mid-Senior level,Consumer Goods Consumer Services Financial Ser...,Full-time,Information Technology Finance Project Management,Posted Date\nPosted 1 week ago\nNumber of view...,A large company based near Madrid is currentl...,data analyst_1811532919
2,data analyst,Madrid,2020-04-20 01:38,1800343808,https://www.linkedin.com/jobs/view/1800343808/,Senior Business Development Manager - Cloud Ec...,Amazon Web Services (AWS),"Madrid, ES",3 weeks,2020-03-30 01:38,False,"{'yes': ['Statistics', 'SQL', 'Microsoft Excel...",Mid-Senior level,Computer Software Information Technology & Ser...,Full-time,Business Development Information Technology,Posted Date\nPosted 3 weeks ago\nNumber of vie...,Description\n\nWould you like to be part of a ...,data analyst_1800343808
3,data analyst,Madrid,2020-04-20 01:38,1712806429,https://www.linkedin.com/jobs/view/1712806429/,Digital Data Scientist,JTI (Japan Tobacco International),"Madrid, ES",4 weeks,2020-03-23 01:38,False,"{'yes': ['Data Visualization', 'Data Analytics...",Entry level,Consumer Goods Food & Beverages Tobacco,Full-time,Engineering Information Technology,Posted Date\nPosted 4 weeks ago\nNumber of vie...,JTI is a leading international tobacco company...,data analyst_1712806429
4,data analyst,Madrid,2020-04-20 01:38,1786901267,https://www.linkedin.com/jobs/view/1786901267/,Data Analytics Manager Madrid,Accenture España,"Madrid, ES",13 hours,2020-04-19 12:38,False,"{'yes': ['Data Visualization', 'Data Analytics...",Mid-Senior level,Information Technology & Services Computer Sof...,Full-time,Business Development Sales,New\nPosted Date\nPosted 13 hours ago\nNumber ...,Accenture Digital offers a comprehensive portf...,data analyst_1786901267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,data engineer,Madrid,2020-04-20 01:38,1808203945,https://www.linkedin.com/jobs/view/1808203945/,Senior Consultant,Board International Sa,"Madrid, ES",2 weeks,2020-04-06 01:38,False,"{'yes': ['Apache Spark'], 'no': ['Hive', 'Hado...",Associate,Information Technology & Services Computer Sof...,Full-time,Consulting Information Technology Sales,Posted Date\nPosted 2 weeks ago\nNumber of vie...,"We are looking for a highly analytical, self-m...",data engineer_1808203945
461,data engineer,Madrid,2020-04-20 01:38,1802827962,https://www.linkedin.com/jobs/view/1802827962/,DevSecOps Engineer,AIRBUS,"Madrid, ES",1 month,2020-03-23 01:38,False,"{'yes': ['Apache Spark'], 'no': ['Hive', 'Hado...",Entry level,Information Technology & Services Computer Sof...,Full-time,Engineering Information Technology,Posted Date\nPosted 1 month ago\nNumber of vie...,Job Description\n\nDescription of the job\n\nF...,data engineer_1802827962
462,data engineer,Madrid,2020-04-20 01:38,1820745753,https://www.linkedin.com/jobs/view/1820745753/,DevSecOps Engineer,AIRBUS,"Madrid, ES",1 week,2020-04-13 01:38,False,"{'yes': ['Apache Spark'], 'no': ['Hive', 'Hado...",Entry level,Information Technology & S

### > Language of each job description:

In [34]:
df_jobs['Job name'].head(10)

0                                Data Analyst Scala H/M
1     Finance Systems Business Analyst - GL/FA/AP/AR...
2     Senior Business Development Manager - Cloud Ec...
3                                Digital Data Scientist
4                         Data Analytics Manager Madrid
6        Grupo CMC: Analista de datos (python) - madrid
7                    Programmatic Trader & Data Analyst
8                    Programmatic Trader & Data Analyst
9                                    Analistas de datos
10                                   Analistas de datos
Name: Job name, dtype: object

In [35]:
detect(df_jobs.loc[6,'Job Description'])

'es'

In [36]:
#New comñumn identifying language
df_jobs['Info Language'] = df_jobs['Job Description'].apply(lambda x: detect(x))

In [37]:
df_jobs['Info Language'].unique()

array(['es', 'en'], dtype=object)

In [38]:
df_jobs['Seniority Level'].unique()

array(['Mid-Senior level', 'Entry level', 'Associate', nan, 'Internship',
       'Director'], dtype=object)

### > Coincidences with definitions:

In [39]:
def coincidencias(job_skills, post_skills):
    c = 0
    for i in job_skills: 
        if re.search(i,post_skills): 
            c+=1
    return c
        

In [40]:
def list_coincidencias(job_skills, post_skills):
    l=[]
    for i in job_skills:
        if re.search(i,post_skills):
            l.append(i)
    return l
        

In [41]:
def clean_text(text):
    text = text.replace('\n',' ')
    text = text.replace('á','a')
    text = text.replace('é','e')
    text = text.replace('í','i')
    text = text.replace('ó','o')
    text = text.replace('ú','u')
    text = text.replace('.',' ')
    text = text.lower()
    
    return text
    

#### jobs >

In [42]:
#jobs_list = list(df_jobs['JOB TITLE'].unique()) #only jobs that searcht has been taken
unique_jobs = list(df_jobs['JOB TITLE'].unique())

In [43]:
#New columns with coindicences DEFINITION vs Job post
coincidences_jobs = []
for j in unique_jobs:
    skills_list = dic_key_words.get(j) #dictionary
    coincidences_jobs.append(f'{j[0:2]} coincidences')
    df_jobs[f'{j[0:2]} coincidences'] = \
    df_jobs['Job Description'].apply(lambda x: coincidencias(skills_list,clean_text(x)))
    df_jobs[f'{j[0:2]} coincidences_list'] = \
    df_jobs['Job Description'].apply(lambda x: list_coincidencias(skills_list,clean_text(x)))

#     df_jobs['DS coincidences'] = \
#     df_jobs['Job info'].apply(lambda x: coincidencias(DS_list,clean_text(x)))
#     df_jobs['DS coincidences_list'] = \
#     df_jobs['Job info'].apply(lambda x: list_coincidencias(DS_list,clean_text(x)))

#     df_jobs['DE coincidences'] = \
#     df_jobs['Job info'].apply(lambda x: coincidencias(DE_list,clean_text(x)))
#     df_jobs['DE coincidences_list'] = \
#     df_jobs['Job info'].apply(lambda x: list_coincidencias(DE_list,clean_text(x)))

#### profile >

In [44]:
df_profile.loc['total skills', 'info']

"['analytical skills', 'bash', 'business analysis', 'business intelligence', 'business process improvement', 'continuous improvement', 'data analysis', 'data analytics', 'data modeling', 'data pipelines', 'data science', 'data visualization', 'engineering', 'engineering - be', 'english', 'español', 'financial analysis', 'git', 'industrial engineering', 'industry knowledge', 'jupyter', 'leadership', 'lean manufacturing', 'machine learning', 'master', 'microsoft excel', 'microsoft office', 'microsoft power bi', 'microsoft power query', 'microsoft project', 'mysql', 'negotiation', 'numpy', 'pandas', 'power pivot', 'process engineering', 'process optimization', 'programming', 'project management', 'python', 'r', 'sap', 'scrum', 'seaborn', 'six sigma', 'spanish', 'sql', 'statistics', 'tableau', 'team management', 'teamwork', 'telco', 'vba', 'visual basic for applications', 'web scraping']"

In [45]:
profile_skills = clean_text(df_profile.loc['skills', 'info'])
#### THIS IS ONLY FOR JUPYTER !!!!!!!!!!!!!!!!!!!!
profile_skills = profile_skills.replace('[','').replace(']','').replace("'","").split(', ')

sorted(profile_skills)

['analytical skills',
 'bash',
 'business analysis',
 'business intelligence',
 'business process improvement',
 'continuous improvement',
 'data analysis',
 'data analytics',
 'data modeling',
 'data pipelines',
 'data science',
 'data visualization',
 'engineering',
 'english',
 'español',
 'financial analysis',
 'git',
 'industry knowledge',
 'jupyter',
 'leadership',
 'lean manufacturing',
 'machine learning',
 'microsoft excel',
 'microsoft office',
 'microsoft power bi',
 'microsoft power query',
 'microsoft project',
 'mysql',
 'negotiation',
 'numpy',
 'pandas',
 'power pivot',
 'process engineering',
 'process optimization',
 'programming',
 'project management',
 'python',
 'r',
 'sap',
 'scrum',
 'seaborn',
 'six sigma',
 'sql',
 'statistics',
 'tableau',
 'team management',
 'teamwork',
 'telco',
 'vba',
 'visual basic for applications',
 'web scraping']

In [46]:
df_jobs['PROFILE coincidences'] = \
df_jobs['Job Description'].apply(lambda x:      coincidencias(profile_skills,clean_text(x)))
df_jobs['PROFILE coincidences_list'] = \
df_jobs['Job Description'].apply(lambda x: list_coincidencias(profile_skills,clean_text(x)))

In [47]:
df_jobs.head(2)

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,...,Employment Type,Job Functions,Job info,Job Description,JOB TITLE_ID,Info Language,da coincidences,da coincidences_list,PROFILE coincidences,PROFILE coincidences_list
0,data analyst,Madrid,2020-04-20 01:38,1821611516,https://www.linkedin.com/jobs/view/1821611516/,Data Analyst Scala H/M,Experis IT,"Madrid, Comunidad de Madrid, España",5 days,2020-04-15 01:38,...,Full-time,Information Technology Consulting Project Mana...,Posted Date\nPosted 5 days ago\nNumber of view...,"Experis IT, somos una compañía líder en soluci...",data analyst_1821611516,es,3,"[ r , scala, big data]",5,"[business intelligence, data analysis, data an..."
1,data analyst,Madrid,2020-04-20 01:38,1811532919,https://www.linkedin.com/jobs/view/1811532919/,Finance Systems Business Analyst - GL/FA/AP/AR...,SystemsAccountants,Greater Madrid Metropolitan Area,1 week,2020-04-13 01:38,...,Full-time,Information Technology Finance Project Management,Posted Date\nPosted 1 week ago\nNumber of view...,A large company based near Madrid is currentl...,data analyst_1811532919,en,0,[],6,"[business analysis, english, git, project mana..."


#### checking results >

In [48]:
check = 5

In [49]:
df_jobs.loc[check ,'Job name']

'Analistas de datos'

In [50]:
dict(df_jobs.loc[check ,'da coincidences':'PROFILE coincidences_list'])

{'da coincidences': 7,
 'da coincidences_list': ['sql',
  'nosql',
  'python',
  ' r ',
  'hadoop',
  'mapreduce',
  'big data'],
 'PROFILE coincidences': 7,
 'PROFILE coincidences_list': ['data science',
  'machine learning',
  'mysql',
  'python',
  'r',
  'scrum',
  'sql']}

In [51]:
clean_text(df_jobs.loc[check ,'Job Description'])

'descripcion del trabajo  detalles del puesto  en grupo invoin estamos buscando analistas de datos para un interesante proyecto estable de gestion tecnologica en madrid   te escuchamos y te hacemos una oferta salario abierto a experiencia: te ayudamos a mejorar tus condiciones actuales  buen entorno laboral, ambiente joven y dinamico y crecimiento profesional  paquete de beneficios sociales (ticket comida, ticket transporte, cheque guarderia, seguro medico privado)  ubicacion: alcobendas (madrid)  puesto estable, jornada completa  horario: lunes a jueves de 8:00h a 17:00h viernes de 8:00h a 15:00h requisitos 3 años de experiencia en la aplicacion de tecnologias en proyectos de analitica avanzada de grandes volumenes de datos e inteligencia artificial  experiencia desarrollando proyectos de ingenieria e inteligencia de datos, coordinando con equipos multidisciplinares y aplicando metodologias agiles  titulacion superior en estadistica, economicas, matematicas, informatica o ingenieria  

### TOP JOB POST >

In [56]:
top = 5

In [83]:
df_top_jobs = df_jobs\
.drop_duplicates(subset='Current Job Id', keep='first')\
.sort_values(by=['PROFILE coincidences'], ascending=False).head(top)\
[['JOB TITLE', 'Job name','Company name','Company location','Posted date','Employment Type',
  'Job html','Multiple Job Post']]
df_top_jobs

,JOB TITLE,Job name,Company name,Company location,Posted date,Employment Type,Job html,Multiple Job Post
232,data scientist,Data Scientist & Project Administrator (SoF),Brambles,"Madrid, ES",1 month,Full-time,https://www.linkedin.com/jobs/view/1772483934/,-DS--
123,data scientist,Consultor Data Scientist,Bip Spain,"Madrid y alrededores, España",1 week,Full-time,https://www.linkedin.com/jobs/view/1793291346/,-DS--
103,data scientist,Data Analyst,Amazon,"Madrid, ES",3 weeks,Full-time,https://www.linkedin.com/jobs/view/1827111451/,-DS--
117,data scientist,Data Analyst,OrbitalAds,"Madrid, ES",1 month,Full-time,https://www.linkedin.com/jobs/view/1748642606/,-DS--
55,data analyst,4P Internal Use Cases Business Data Analyst en...,Telefonica,"Madrid, ES",1 month,Internship,https://www.linkedin.com/jobs/view/1768578002/,DA-DS--


In [58]:
df_top_jobs['Job html'] = df_top_jobs['Job html'].apply(lambda x: f'<a href={x}>link</a>')

In [59]:
df_top_jobs.head(10)

,JOB TITLE,Job name,Company name,Company location,Posted date,Employment Type,Job html
234,data scientist,Data Scientist & Project Administrator (SoF),Brambles,"Madrid, ES",1 month,Full-time,<a href=https://www.linkedin.com/jobs/view/177...
124,data scientist,Consultor Data Scientist,Bip Spain,"Madrid y alrededores, España",1 week,Full-time,<a href=https://www.linkedin.com/jobs/view/179...
104,data scientist,Data Analyst,Amazon,"Madrid, ES",3 weeks,Full-time,<a href=https://www.linkedin.com/jobs/view/182...
118,data scientist,Data Analyst,OrbitalAds,"Madrid, ES",1 month,Full-time,<a href=https://www.linkedin.com/jobs/view/174...
56,data analyst,4P Internal Use Cases Business Data Analyst en...,Telefonica,"Madrid, ES",1 month,Internship,<a href=https://www.linkedin.com/jobs/view/176...


## render dataframe as html
html = df_top_jobs.to_html(escape=False)

#write html to file
if platform.system() == 'Linux':
    text_file = open(f"../data/results/top_{top}_jobs_in_{location}.html", "w")
    text_file.write(html)
    text_file.close()
    #text_file.close()
    
    
elif platform.system() == 'Windows':
    text_file = open(f"..\\data\\results\\top_{top}_jobs_in_{location}.html", "w")
    text_file.write(html)
    #text_file.close()

#### checking results >

In [ ]:
df_jobs[df_jobs['Current Job Id']==1682047755]

## Using NLP

In [84]:
nlp_en = spacy.load('en_core_web_md')
nlp_es = spacy.load('es_core_news_md')

In [85]:
df_jobs.columns

Index(['JOB TITLE', 'LOCATION', 'SEARCH DATETIME', 'Current Job Id',
       'Job html', 'Job name', 'Company name', 'Company location',
       'Posted date', 'Estimated post date', 'Easy apply', 'Skills match',
       'Seniority Level', 'Industry', 'Employment Type', 'Job Functions',
       'Job info', 'Job Description', 'JOB TITLE_ID', 'Info Language',
       'da coincidences', 'da coincidences_list', 'PROFILE coincidences',
       'PROFILE coincidences_list', 'Multiple Job Post'],
      dtype='object')

In [86]:
unique_jobs[0].split(" ")[0][0].upper() + unique_jobs[0].split(" ")[1][0].upper()

'DA'

In [87]:
example = "data hola kase"
nuevo_key = job_key_generator(example)
nuevo_key

'DHK'

In [88]:
unique_jobs

['data analyst', 'data scientist', 'data engineer']

In [89]:
unique_languages = list(df_jobs['Info Language'].unique())
unique_languages

['es', 'en']

In [90]:
location = df_jobs['LOCATION'].unique()[0].lower()
location

'madrid'

In [91]:
black_list = \
[location,
 'empresa','work','skill',
'parir','comer','timar',
 'job','industry', 'type','employment', 'full','functions','seniority','level',
 'company','experience','help','time','new','role','use','year','enable',
]

In [92]:
dict_corpus = {}
dict_filtered_corpus = {}
dict_filtered_tokens = {}
dict_word_freq = {}
dict_common_words_100 = {}

dict_filtered_corpus_NN = {}
dict_filtered_tokens_NN = {}
dict_word_freq_NN = {}
dict_common_words_100_NN = {}

In [ ]:
%%time
for job in unique_jobs:
    

    job_key_s = job_key_generator(job)
    
    for lang in unique_languages:
        
        #CORPUS
        filter_job  = df_jobs['JOB TITLE'] == job
        filter_lang = df_jobs['Info Language'] == lang
        corpus = df_jobs['Job Description'][filter_job & filter_lang].str.cat(sep=" || ")
        
        dict_corpus.update( {f'{job_key_s}_{lang}' : corpus} )
        
        
        #TOKENS
        if   lang == 'en':
            tokens = nlp_en(corpus)
            STOP_WORDS = SW_en
        elif lang == 'es':
            tokens = nlp_es(corpus)
            STOP_WORDS = SW_es
        
        filtered_corpus = ""
        filtered_tokens = []
        #NOUNS
        filtered_corpus_NN = ""
        filtered_tokens_NN = []

        for word in tokens:
            lemma = word.lemma_.lower().strip()
            NN    = word.pos_.lower().strip()

            if lemma not in STOP_WORDS and re.search('^[a-z]+$', lemma) and lemma not in black_list and len(lemma)>1:
                    filtered_corpus += lemma + " "
                    filtered_tokens.append(lemma)
                    
                    #NOUNS
                    if word.pos_ == 'NOUN':
                        filtered_corpus_NN += lemma + " "
                        filtered_tokens_NN.append(lemma)
        
        dict_filtered_corpus.update( {f'{job_key_s}_{lang}': filtered_corpus} )
        dict_filtered_tokens.update( {f'{job_key_s}_{lang}': filtered_tokens} )
        
        #NOUNS
        dict_filtered_corpus_NN.update( {f'{job_key_s}_{lang}': filtered_corpus_NN} )
        dict_filtered_tokens_NN.update( {f'{job_key_s}_{lang}': filtered_tokens_NN} )
        
        #100 COMMON WORDS
        word_freq = Counter(filtered_tokens)
        common_words_100 = word_freq.most_common(100)
        dict_word_freq.update({f'{job_key_s}_{lang}': word_freq})
        dict_common_words_100.update({f'{job_key_s}_{lang}': common_words_100})
        
        #NOUNS
        word_freq_NN = Counter(filtered_tokens_NN)
        common_words_100_NN = word_freq_NN.most_common(100) 
        dict_word_freq_NN.update({f'{job_key_s}_{lang}': word_freq_NN})
        dict_common_words_100_NN.update({f'{job_key_s}_{lang}': common_words_100_NN})
        
        if platform.system() == 'Linux':
            with open(f'../data/results/dict_word_freq{job_key_s}_{lang}.json', 'w') as fp:
                json.dump(dict_word_freq, fp)
        elif platform.system() == 'Windows':
            with open(f'..\\data\\results\\dict_word_freq{job_key_s}_{lang}.json', 'w') as fp:
                json.dump(dict_word_freq, fp)

#### checking results >

In [ ]:
job_lang = 'DS_es'

In [ ]:
dict_corpus.keys()

In [ ]:
dict_corpus.get(job_lang)[400:600]

In [ ]:
dict_filtered_tokens.keys()

In [ ]:
dict_filtered_tokens.get(job_lang)[600:610]

In [ ]:
dict_word_freq.keys()

In [ ]:
dict_common_words_100.keys()

In [ ]:
dict_common_words_100.get(job_lang)[0:20]

In [ ]:
dict_common_words_100_NN.get(job_lang)[0:40]

## CLOUD 

In [ ]:
unique_jobs

In [ ]:
unique_languages

In [ ]:
#def cloud_word(unique_jobs):

for j in unique_jobs:
    

    """
    if platform.system() == 'Linux':
        try:
            mask = np.array(Image.open(f"../data/raw/{j}.png"))
        except:
            mask = np.array(Image.open(f"../data/raw/{j}.png"))

    elif platform.system() == 'Windows':
        try:
            mask = np.array(Image.open(f"..\\data\\raw\\{j}.png"))
        except:
            mask = np.array(Image.open(f"..\\data\\raw\\{j}.png"))

    image_colors = ImageColorGenerator(mask)
    """

    job_key_w = job_key_generator(j)
    plot_rows = len(unique_languages)

    fig , axes = plt.subplots(ncols=1, nrows=plot_rows)

    ax = 0
    for lang in unique_languages:
        print(f'{job_key_w}_{lang}')
        plt.axis("off")
        word_corpus = dict_filtered_corpus.get(f'{job_key_w}_{lang}')
        wordcloud = \
        WordCloud(max_font_size=100, max_words=100, background_color="white")\
                .generate(word_corpus)
        #, mask=mask)\
        axes[ax].axis("off")
        axes[ax].set_title(f'{job_key_w}_{lang}')
        axes[ax].imshow(wordcloud, interpolation='bilinear')

        ax+=1
    plt.axis("off")  
    fig.get_figure()
    plt.show()


    if platform.system() == 'Linux':
        fig.savefig(f'../data/results/worldcloud_{job_key_w}_{location}.png', quality =95, dpi=300)
        #wordcloud.to_file(f'../data/results/worldcloud_{job_lang}_{location}.png')

    elif platform.system() == 'Windows':
        fig.savefig(f'..\\data\\results\\worldcloud_{job_key_w}_{location}.png', quality =95, dpi=300)
        #wordcloud.to_file(f'..\\data\\results\\worldcloud_{job_lang}_{location}.png')
        
    

In [ ]:
#cloud_word(unique_jobs)

In [ ]:
# Display the generated image:
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.show()

## Using ML

***
# TESTING

### Counting frequent words

In [ ]:
c = re.findall('\{(.*)\}',str(dict_common_words.get(job_lang)))[0].split(',')
c

In [ ]:
word_freq = Counter(dict_filtered_tokens.get('DA_en'))
type(word_freq)

In [ ]:
common_words = word_freq.most_common
type(common_words)

In [ ]:
common_words

### masks in CLOUD

In [ ]:
if platform.system() == 'Linux':
    mask = np.array(Image.open("../data/raw/plot.png"))
    
elif platform.system() == 'Windows':
    mask = np.array(Image.open("../data/raw/monitor.png"))

    
image_colors = ImageColorGenerator(mask)

In [ ]:
wordcloud = WordCloud(max_font_size=100, max_words=100, mode="RGBA", background_color="white", mask=mask)\
            .generate(dict_filtered_corpus.get(job_lang))

In [ ]:
plt.figure(figsize=[7,7])
plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation='bilinear')
plt.axis("off")
plt.show()